# Curso Transfer Learning

<img src="https://yaelmanuel.com/wp-content/uploads/2021/12/platzi-banner-logo-matematicas.png" width="500px">

---

## Creando nuestro propio Finetuning de un modelo con OpenAI 😎 🌮

En este lab aprenderás:

* [Open AI API](https://openai.com/api/)
* Descargar un dataset, prepararlo, realizar finetuning y guardarlo.


### 1) Descarga del dataset 🤓

Utilizaremos un conjunto de datos de recetas de cocina de Latinoamérica.
<br>Para más detalle acá se puede ver el dataset de Hugging Face: [somosnlp/recetas-cocina](https://huggingface.co/datasets/somosnlp/recetas-cocina).


In [ ]:
import pandas as pd

In [ ]:
url_dataset = "hf://datasets/somosnlp/recetas-cocina/dataset.csv"
df = pd.read_csv(url_dataset)

In [ ]:
df.head(3)

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25


### 2) Preparación de la data 👌

#### 2.1) Instalamos las dependencias 🙌

In [ ]:
!pip install --upgrade openai

In [ ]:
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
import json

import pandas as pd

from openai import OpenAI

#### 2.2) Acondicionar columnas 👀

Basados en la [documentación oficial de OpenAI](https://platform.openai.com/docs/guides/fine-tuning#example-format) debemos darle una estructura recomendada a nuestra data.

Algo así como: Lo que pregunta el usuario + Lo que responde el modelo.

`{"user": "Receta para arepas:", "assistant": "Ingredientes: 1 taza de harina de arepa blanca... Instrucciones: Mezcla la harina, el agua y la sal..."}`

Tener cuidado con agregar a "\n" y "\r\n" las columnas "ingredients" y "steps" para evitar problemas con las estructura del JSONL.

In [ ]:
# Reemplazar "\r\n" y "\n" por " ."
df = df.replace(to_replace={r'\r\n': ' .', r'\n': ' .'}, regex=True)

In [ ]:
df.head(3)

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla .1...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel .1 cucharada de ...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada .1 pimientón rojo fin...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25


In [ ]:
df_recetas = pd.DataFrame({
    'user': 'Receta para: ' + df['title'],
    'assistant': 'Ingredientes: ' + df['ingredients'] + '. Instrucciones: ' + df['steps']
})

In [ ]:
df_recetas.head(3)

,user,assistant
0,Receta para: Arepas de Queso,Ingredientes: 1 taza de harina de arepa blanca...
1,Receta para: Sudado de Pollo,Ingredientes: 8 muslos de pollo sin la piel .1...
2,Receta para: Sancocho Trifásico,Ingredientes: 1 taza de cebolla picada .1 pimi...


#### 2.3) Creación de JSONL 🔍

[OpenAI API](https://platform.openai.com/docs/api-reference/files/create) utiliza para la carga de archivos el formato JSONL.

**JSONL** es una colección de objetos JSON donde cada línea representa un registro independiente.

Esto hace que sea fácil estructurar datos complejos con jerarquías y metadatos asociados, ideal para entradas y salidas de modelos.

**Importante:** Hay un límite del tamalo de archivo que podemos cargar a OpenAI, según la [documentación actual](https://platform.openai.com/docs/guides/fine-tuning#size-limits-and-large-uploads) es de 512 MB.

In [ ]:
jsonl_path = 'recetas_latinoamericanas.jsonl'

In [ ]:
with open(jsonl_path, 'w', encoding='utf-8') as file:
   # Itera solo sobre las primeras 15 filas del dataframe
    for _, row in df_recetas.head(15).iterrows():
        message = {
            "messages": [
                {"role": "system", "content": "Eres un experto culinario especializado en recetas latinoamericanas."},
                {"role": "user", "content": row["user"]},
                {"role": "assistant", "content": row["assistant"]}
            ]
        }
        json.dump(message, file, ensure_ascii=False)
        file.write('\n')  # Escribe un salto de línea para cumplir con el formato JSONL

print(f"Archivo {jsonl_path} creado correctamente.")

Archivo recetas_latinoamericanas.jsonl creado correctamente.


El archivo recetas_latinoamericanas.jsonl pesa aproximadamente 20 KB.

### 3) Carga del archivo JSONL a OpenAI 💪

In [ ]:
# Load the environment variables
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
client = OpenAI()

response_loading = client.files.create(
  file=open(jsonl_path, "rb"),
  purpose="fine-tune"
)

Pueden ver más información en el [Dashboard de OpenAI](https://platform.openai.com/finetune/).

In [ ]:
print(response_loading)

FileObject(id='file-Udu3ucguHnoRUHjK4jRTpn', bytes=18153, created_at=1736960333, filename='recetas_latinoamericanas.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [ ]:
# Acceder al ID del archivo creado
file_loaded_id = response_loading.id

print(f"El ID del archivo creado es: {file_loaded_id}")

El ID del archivo creado es: file-Udu3ucguHnoRUHjK4jRTpn


### 4) Finetuning 😨

Dentro de la [documentación oficial](https://platform.openai.com/docs/guides/fine-tuning#which-models-can-be-fine-tuned) se muestra a qué modelos podemos aplicarle finetuning.

In [ ]:
modelo_openai = "gpt-4o-mini-2024-07-18"

In [ ]:
client = OpenAI()

response_finetuning = client.fine_tuning.jobs.create(
    training_file=file_loaded_id,
    model=modelo_openai,
)

En caso de quere ver más hiperparámetros que se puedem ajustar, hay una [sección oficial en la documentación de OpenAI](https://platform.openai.com/docs/guides/fine-tuning#which-models-can-be-fine-tuned).

Para obtener el id del job:

In [ ]:
job_id = response_finetuning.id
print(f"El job ID del modelo es: {job_id}")

El job ID del modelo es: ftjob-ROO24HnkmVbr9mvcegZAnnt5


In [ ]:
job_details = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model = job_details.fine_tuned_model

print(f"Nombre del modelo afinado: {fine_tuned_model}")

Nombre del modelo afinado: ft:gpt-4o-mini-2024-07-18:personal::Aq1BTT8A


Si todavía no está listo el finetuning les va a devolver None.

### 5) Hacer Predicciones en Producción 🤙

Función de inferencia / predicción

In [ ]:
role_assistant_prompt = "Eres un asistente de cocina"

In [ ]:
client = OpenAI()

def ask_model(question):
  completion = client.chat.completions.create(

    # Cargar nuestro modelo afinado
    model=fine_tuned_model,

    messages=[
        {"role": "system", "content": role_assistant_prompt},
        {"role": "user", "content": question}
    ]
)

  # Mostrar objeto completo
  print(completion.choices[0].message)
  # Mostrar solo contenido de la respuesta
  #print(completion.choices[0].message.content)

Ejemplos de uso

In [ ]:
question =  "Me pasas los ingredientes para hacer unas arepas de queso"
ask_model(question)

ChatCompletionMessage(content='Claro. Aquí tienes una receta de arepas de queso. Ingredientes: 2 tazas de harina de maíz blanco o amarillo, 2 tazas de agua tibia, 2 cucharadas de mantequilla, derretida, 1 cucharadita de sal, ½ taza de queso mozzarella o queso blanco rallado, .Instrucciones: Mezclar la harina de maíz, el agua caliente, la mantequilla y la sal en un tazón. Revolver con un tenedor y dejar reposar durante unos 5 a 10 minutos. Asegúúrese de sacudir con un tenedor para que la harina de maíz no se apelmace. .Añadir el queso, muelas con las manos y amasar para que se mezcle. .Forma las arepas en bolitas y luego haga una torta gruesa. .Coloque la mantequilla en una sartén antiadherente y caliente a fuego medio. Coloque las arepas en la sartén y cocine durante aproximadamente 3 minutos de cada lado, hasta que estén doradas.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [ ]:
question = "Quiero aprender a hacer Sancocho Trifásico, me pasas los ingredientes y cómo se hacee"
ask_model(question)

ChatCompletionMessage(content='Ingredientes 1 cola de res en trozos . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  .1 cucharadita de aceite vegetal . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .1 diente de ajo picado . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .5 tazas de caldo de carne . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .5 papas pequeñas peladas y cortadas por la mitad . . . . . . . . . . . . . . . . .2 mazorca de maiz cortadas en rodajas . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .3 tazas de yuca pelada y cortada en trozos de 2 pulgadas . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [ ]:
question = "receta para una pizza italiana"
ask_model(question)

ChatCompletionMessage(content='Ingredientes: ½ libra de harina para hacer la masa .½ taza de agua tibia .1 cucharadita de azúcar .2 cucharadas de aceite de oliva .1 cucharadita de sal .1 cucharadita de levadura activa .½ taza de salsa de tomate .8 onzas de queso mozzarella .Toppings de su elección. Instrucciones: Combine el agua tibia, el azúcar y la levadura en un tazón pequeño, mezcle y deje reposar durante 5 minutos. .En un tazón grande, mezcle la harina, la sal y haga un hueco en el centro. Verter la mezcla de levadura y el aceite de oliva, y mezcle con un tenedor hasta que la masa se forme. .Amasar la masa sobre una superficie enharinada durante unos 7 minutos hasta que esté suave y elástica. Coloque la masa en un tazón engrasado, tape y deje que suba hasta que doble su tamaño, aproximadamente 1 hora. .Divida la masa en 2 porciones iguales, forme cada porción en una bola y dejar que suban unos 15 minutos. .Estire cada barrio con un rodillo hasta obtener una pizza de forma redonda 

In [ ]:
question = "Cuántos muslos de pollo lleva el Sudado de Pollo? Fijate en la lista de ingredientes y decime"
ask_model(question)

In [ ]:
question = "me pasas la receta del sudado de pollo"
ask_model(question)

In [ ]:
question = "receta para sudado de pollo"
ask_model(question)

### 6) Conclusiones

- Aprender sobre los distintos objetos y métodos que nos ofrece OpenAI API.

- Realizar el proceso completo de aplicar finetuning a un modelo de OpenAI.

- Aprender manejo de memoria, tipos de datos y procesos de entrenamiento.

<br>
<br>
<br>

---

<br>
<br>


<img src="https://static.platzi.com/media/avatars/platziteam_8cfe6fc7-1246-4c9a-9f5d-d10d467443ee.png" width="100px">

[Platzi](https://platzi.com/) 🚀

